In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('samsung_price.xlsx')

In [3]:
stock_list = pd.read_html('https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

In [4]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.03.23,70500,200,70600,71200,70300,12332242
1,2022.03.22,70300,400,69900,70500,69900,9402666
2,2022.03.21,69900,800,70900,71000,69900,11169002
3,2022.03.18,70700,500,70600,70900,70200,14410038
4,2022.03.17,71200,800,71200,71800,70900,17646315


In [5]:
stock_list['종목코드'] = stock_list['종목코드'].map('{:06d}'.format)
stock_list.sort_values(by='종목코드', inplace=True)
stock_list.reset_index(drop=True, inplace=True)
stock_list.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,동화약품,000020,의약품 제조업,"의약품 제조,판매,수출입",1976-03-24,12월,유준하,http://www.dong-wha.co.kr,서울특별시
1,KR모터스,000040,그외 기타 운송장비 제조업,"이륜차(오토바이) 제조,도매",1976-05-25,12월,노성석,http://www.krmotors.com,경상남도
2,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
3,메리츠화재,000060,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
4,삼양홀딩스,000070,기타 금융업,"지주회사,경영자문컨설팅",1968-12-27,12월,"엄태웅, 윤재엽",http://www.samyang.com,서울특별시


In [28]:
code_list = ['323410']

In [29]:
from bs4 import BeautifulSoup
import requests

In [34]:
def get_df(code_list):
    df = pd.DataFrame()
    for i in range(len(code_list)):
        code = code_list[i]
        url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
        req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        html = BeautifulSoup(req.text, "lxml")
        pgrr = html.find('td', class_='pgRR')
        s = pgrr.a['href'].split('=')
        last_page = s[-1]
        for page in range(1, int(last_page)+1):
            req = requests.get(f'{url}&page={page}', headers={'User-agent': 'Mozilla/5.0'})
            df = pd.concat([df, pd.read_html(req.text, encoding = 'euc-kr')[0]], ignore_index=True)
            df.dropna(inplace=True)
            # df = df.loc[::-1]
            df.reset_index(drop=True, inplace=True)
        
        # 전일대비 종가 등락률 계산
        close = df['종가']
        daily_rate = (close / close.shift(-1)-1)* 100
        df['일간변동률(%)'] = daily_rate
        
        # 전일대비 거래량 등락률 계산
        volume = df['거래량']
        volume_rate = (volume / volume.shift(-1)-1) * 100
        df['거래량증감률(%)'] = volume_rate
        
        # RSI 계산을 위한 전일비 등락값
        # 기존 전일비 값은 +, -의 구분이 없음
        diff = close - close.shift(-1)
        df['전일비등락'] = diff
        
        # RSI (14일 기준) 계산
        df = df[::-1]
        u = np.where(df['종가'].diff(1) > 0, df['종가'].diff(1), 0)
        d = np.where(df['종가'].diff(1) < 0, df['종가'].diff(1) *(-1), 0)
        df['up'] = u
        df['down'] = d
        au = df['up'].rolling(window=14).mean()
        ad = df['down'].rolling(window=14).mean()
        rsi = au / (ad+au) * 100
        #rs = au / ad
        #rsi = 100.0 - (100.0 / (1.0+rs))
        df['RSI'] = rsi
        
    return df

In [35]:
test = get_df(code_list)

In [36]:
# 데이터가 없는 행 일괄 삭제
# test.dropna(inplace = True)
# 인덱스 재배열
# test.reset_index(drop=True, inplace=True)

In [37]:
test

,날짜,종가,전일비,시가,고가,저가,거래량,일간변동률(%),거래량증감률(%),전일비등락,up,down,RSI
152,2021.08.06,69800.0,16100.0,53700.0,69800.0,51000.0,59402373.0,NaN,NaN,NaN,0.0,0.0,NaN
151,2021.08.09,78500.0,8700.0,69800.0,89100.0,69000.0,44833696.0,12.464183,-24.525412,8700.0,8700.0,0.0,NaN
150,2021.08.10,71400.0,7100.0,81800.0,85600.0,68000.0,26495122.0,-9.044586,-40.903552,-7100.0,0.0,7100.0,NaN
149,2021.08.11,74400.0,3000.0,71200.0,77500.0,68100.0,21592734.0,4.201681,-18.502983,3000.0,3000.0,0.0,NaN
148,2021.08.12,73800.0,600.0,73600.0,75400.0,72700.0,5878196.0,-0.806452,-72.776972,-600.0,0.0,600.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022.03.18,52800.0,700.0,52300.0,52800.0,51800.0,834273.0,1.343570,-33.154898,700.0,700.0,0.0,80.102041
3,2022.03.21,50500.0,2300.0,52800.0,53000.0,50200.0,1389894.0,-4.356061,66.599422,-2300.0,0.0,2300.0,62.222222
2,2022.03.22,50900.0,400.0,50300.0,51200.0,49900.0,764010.0,0.792079,-45.031060,400.0,400.0,0.0,62.719298
1,2022.03.23,52300.0,1400.0,51200.0,52300.0,51100.0,1190716.0,2.750491,55.850840,1400.0,1400.0,0.0,67.322835


In [39]:
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()